In [4]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [5]:
import tensorflow as tf
from tensorflow import keras

# Image parameters
img_h = 16
img_w = 16
channels = 3

# Class names
class_names = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]
class_names_tf = tf.constant(class_names)  # ✅ FIX

# ------------------------------------------------------
# Image loading
# ------------------------------------------------------
def read_decode(file_name, resize_dim):
    img_bytes = tf.io.read_file(file_name)
    img = tf.image.decode_jpeg(img_bytes, channels=channels)  # ✅ force RGB
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, resize_dim)
    return img

# ------------------------------------------------------
# CSV parsing
# ------------------------------------------------------
def parse_csvline(csv_line):
    record_default = ["", ""]
    file_name, label_string = tf.io.decode_csv(csv_line, record_default)

    img = read_decode(file_name, [img_h, img_w])

    # ✅ FIXED LABEL ENCODING
    label = tf.argmax(
        tf.cast(tf.equal(class_names_tf, label_string), tf.int32)
    )

    return img, label

# ------------------------------------------------------
# Datasets
# ------------------------------------------------------
train_dataset = (
    tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
    .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(8)
    .prefetch(tf.data.AUTOTUNE)
)

eval_dataset = (
    tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
    .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(8)
    .prefetch(tf.data.AUTOTUNE)
)

# ------------------------------------------------------
# Model
# ------------------------------------------------------
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(img_h, img_w, channels)),
    keras.layers.Dense(
        128,
        kernel_regularizer=keras.regularizers.l2(0.01),
        use_bias=False
    ),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(class_names), activation="softmax")
])

# ------------------------------------------------------
# Compile
# ------------------------------------------------------
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

# ------------------------------------------------------
# Train
# ------------------------------------------------------
model.fit(
    train_dataset,
    validation_data=eval_dataset,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


    412/Unknown 15s 33ms/step - accuracy: 0.2551 - loss: 4.2179

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.2552 - loss: 4.2170 - val_accuracy: 0.3973 - val_loss: 3.4860
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 0.3556 - loss: 3.6567 - val_accuracy: 0.4324 - val_loss: 3.2259
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 12s 29ms/step - accuracy: 0.3991 - loss: 3.3870 - val_accuracy: 0.4730 - val_loss: 3.0895
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.4340 - loss: 3.1839 - val_accuracy: 0.4892 - val_loss: 2.9596
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - accuracy: 0.4666 - loss: 3.0004 - val_accuracy: 0.4459 - val_loss: 2.9077
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 13s 31ms/step - accuracy: 0.4850 - loss: 2.8428 - val_accuracy: 0.4459 - val_loss: 2.7758
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.4908 - loss: 2.7135 - val_accuracy: 0.5000 - val_loss: 2.6373
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 13s 32ms/step - accuracy: 0.5146 - loss: 2.5655 - val_accurac